In [1]:
from __future__ import annotations
from cProfile import label
from email.mime import image
import os
import torch
import pandas as pd
import numpy as np
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import json
import torch
import random
import math
import utils
from utils.model import get_model_object_detection
from utils.utils import collate_fn
from pycocotools.coco import COCO
from utils.engine import train_one_epoch, evaluate
from utils.helper import get_transform


In [2]:
#plt.switch_backend('Tkagg')
root_dir = '/opt/data/team/hien/data/raw'

In [3]:
class BPlanDataset(Dataset):
    def __init__(self, root, transforms=None):

        self.root = root

        json_file = open(
            self.root + "/BPlan_Berlin_Planzeichen_resized.json")
        self.obj = json.load(json_file)
        self.coco = COCO(self.root + "/BPlan_Berlin_Planzeichen_resized.json")
        self.transforms = transforms
        self.category_list = self.get_category_list()
        #print(self.category_list)
        #print(len(self.category_list))
        self.category_id_list = sorted(self.map_cats_to_cat_id().values())
        #print(self.category_id_list)
        #print(len(self.category_id_list))
        self.image_ids = list(sorted(self.get_img_ids_by_cat()))
        #print(self.image_ids)
        #print(len(self.image_ids))
        self.cat_map = self.map_cat_to_internal_cat_ids()
        #print(self.cat_map)

    def get_category_list(self):
        category_list = []
        for cat in self.obj["categories"]:
            category_list.append(cat["name"])
        return category_list

    def map_cats_to_cat_id(self):
        cats_to_cat_id = {}
        for cat_id in self.coco.cats.keys():
            cat_name = self.coco.cats[cat_id]['name']
            cat_id = self.coco.cats[cat_id]['id']
            if cat_name in self.category_list:
                cats_to_cat_id[cat_name] = cat_id
        return cats_to_cat_id

    def get_img_ids_by_cat(self):
        ids = []
        for img_id in self.coco.imgs.keys():
            cats = set(self.coco.imgs[img_id]['category_ids'])
            common_cats = cats.intersection(set(self.category_id_list))
            if len(common_cats) != 0:
                ids.append(img_id)
        return ids 

    def map_cat_to_internal_cat_ids(self):
        map_cat_to_ids = {}
        for i in range(1, len(self.category_id_list) + 1):
            map_cat_to_ids[self.category_id_list[i - 1]] = i
        return map_cat_to_ids

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        coco = self.coco
        img_id = self.image_ids[idx]
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=self.category_id_list)
        anns = coco.loadAnns(ann_ids)
        root = self.root
        
        img_filename = coco.loadImgs(img_id)[0]['file_name']
        image_path = root + "/files/cropped_dataset/" + img_filename

        img = Image.open(os.path.join(self.root, image_path)).convert('RGB')

        num_objs = len(anns)

        boxes = []
        areas = []
        labels = []
        for i in range(num_objs):
            x_min = anns[i]['bbox'][0]
            y_min = anns[i]['bbox'][1]
            x_max = x_min + anns[i]['bbox'][2]
            y_max = y_min + anns[i]['bbox'][3]
            boxes.append([x_min, y_min, x_max, y_max])
            areas.append(anns[i]['area'])
            cat = anns[i]['category_id']
            labels.append(self.cat_map[cat])
       
    

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        img_id = torch.tensor([img_id])

        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = img_id
        target["area"] = areas
        target["iscrowd"] = iscrowd


        if self.transforms is not None:
            img, target = self.transforms(img, target)
        

        return img, target

In [4]:
def collate_fn(batch):
    return tuple(zip(*batch))

def train_test_split(dataset, test_set_proportion):
    indices = list(range(len(dataset)))
    random.shuffle(indices)
    split_index = math.floor(len(indices) * test_set_proportion)

    train = torch.utils.data.Subset(dataset, indices[:-split_index])
    test = torch.utils.data.Subset(dataset, indices[-split_index:])

    return train, test

In [5]:
dataset = BPlanDataset(root_dir, get_transform(train=False))

dataset_train, dataset_test = train_test_split(
        dataset, 0.2
    )


print(len(dataset_train), len(dataset_test))

data_loader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_class =  len(dataset.category_list) +1

model = get_model_object_detection(num_class)
model.to(device)






loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
428 106


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [6]:

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
# lr value changed from 0.005 to 0.0005
optimizer = torch.optim.SGD(params, lr=0.0005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# train it for 10 epochs
num_epochs = 1


if __name__ == '__main__':
    for epoch in range(num_epochs):

        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader_train,
                        device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()


        if epoch == 0:
            %%capture cap_out --no-stderr
            evaluate(model, data_loader_test, device=device)

        else:
            # evaluate on the test dataset
            evaluate(model, data_loader_test, device=device)



/home/pi/.local/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [  0/214]  eta: 0:02:44  lr: 0.000003  loss: 91.2326 (91.2326)  loss_classifier: 1.2908 (1.2908)  loss_box_reg: 0.0001 (0.0001)  loss_objectness: 18.6948 (18.6948)  loss_rpn_box_reg: 71.2469 (71.2469)  time: 0.7702  data: 0.2549  max mem: 2434
Epoch: [0]  [ 10/214]  eta: 0:01:34  lr: 0.000026  loss: 17.8266 (26.4478)  loss_classifier: 1.2323 (1.2244)  loss_box_reg: 0.0467 (0.0674)  loss_objectness: 7.6551 (7.8031)  loss_rpn_box_reg: 7.6946 (17.3530)  time: 0.4651  data: 0.0279  max mem: 3588


KeyboardInterrupt: 

In [ ]:
PATH_txt = "/opt/data/team/hien/models/model_01_03_100_" + str(epoch) + ".txt"
out = cap_out.stdout

with open(PATH_txt, 'w') as f:
    f.write(out)

In [ ]:
PATH = "/opt/data/team/hien/models/model_01_03_100.pt"
torch.save(model.state_dict(), PATH)

In [ ]:
PATH = "/opt/data/team/hien/models/model_01_03_100.pt"
filename = "test_mid.png"
pred_path= root_dir + "/files/prediction/" + filename
#pred_path= '/opt/data/d59/storage6/basic_data/coco_data/BPlan_Berlin_Planzeichen/47150__mdb-bamarzahnhellersdorf-stadtplanung-festgesetb-xxi_32a_abzeichnung.png'

model.eval()
device = torch.device('cpu')
model.load_state_dict(torch.load(PATH))
model.to(device)
#get item
print(num_class)
image= Image.open(pred_path).convert('RGB')
img_np = cv2.imread(pred_path)


convert_tensor = transforms.ToTensor()
image_tensor= convert_tensor(image)


#generate prediction
with torch.no_grad():

    
    prediction = model([image_tensor.to(device)])
    print("Hier ist die Prediction")

    pred = prediction[0]['boxes']
    print(len(pred))
    pred_img = pred.cpu().numpy()

 
    for i in range(len(pred_img)-1):
        x = pred_img[i][0]
        y = pred_img[i][1]
        x2 = pred_img[i][2]
        y2 = pred_img[i][3]
        
        startpoint = (int(x),int(y))
        endpoint= (int(x2), int(y2))
        
        img_np = cv2.rectangle(img_np, startpoint, endpoint, (0,0,255), 2)
    
    directory = root_dir + "/files/model_100_05_05_pred/"
    os.chdir(directory)
    cv2.imwrite("pred_"+filename, img_np)

    
    
print("done")

In [ ]:
%tb